# Measurements type Loading

Notebook to load Measurements type data into the databse

## Imports

In [ ]:
import psycopg2
from tabulate import tabulate
#import a folder in the parent directory
import sys
sys.path.append('../')
import dbtools.dbtools as qrs

## Connection

In [ ]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connected to the database


## Measurement type data

We have to add the parameters of the sample.

The name is a compulsory parameter, it has to be always filled.

The description is not compulsory but is recommended.

In [ ]:
name = 'XCT_2024'

description = 'First XCT measurements, 2024 metodology.'

main_parameters = {'name': name, 'description': description}


Then metadata parameters are optional and can vary depending on the measurement type so its loading to the database is more dinamic.

The metadata list can be empty.

To load metadata parameters we have to fill a list with dictionaries, the dictionaries may have this parameters:

1. key: The name of the parameter

2. value: The value of the parameter

3. type: The units of the parameter

Here we include an example to save the technique, Ultrasound Pulse Echo which type is nominal

metadata_parameters.append({'key':'technique', 'value':'Ultrasound Pulse Echo', 'type':'nominal' })


In [5]:

# metadata

#for each parameter of the measurement a metadata has to be created

metadata_parameters = []

#technique
metadata_parameters.append({'key':'technique', 'value':'X Ray Computed Tomography', 'type':'nominal'})

#voltage
metadata_parameters.append({'key':'voltage', 'value':'50', 'type':'kV'})

#current
metadata_parameters.append({'key':'current', 'value':'90', 'type':'uA'})

#resolution
metadata_parameters.append({'key':'resolution', 'value':'20', 'type':'um'})

#frames
metadata_parameters.append({'key':'frames', 'value':'5', 'type':'cardinal'})

#exposure time
metadata_parameters.append({'key':'exposure time', 'value':'0.2', 'type':'s'})

#projections
metadata_parameters.append({'key':'projections', 'value':'3300', 'type':'cardinal'})

#target
metadata_parameters.append({'key':'target', 'value':'W', 'type':'nominal'})

#filter
metadata_parameters.append({'key':'filter', 'value':'LE1', 'type':'nominal'})

#binning
metadata_parameters.append({'key':'binning', 'value':'1', 'type':'cardinal'})

#equipment
metadata_parameters.append({'key':'equipment', 'value':'Versa', 'type':'nominal'})



## Load into the table

In [6]:
print('Parameters to be inserted: ')
for key, value in main_parameters.items():
    print(f"    -- {key}: {value}")

Parameters to be inserted: 
    -- name: XCT_2024
    -- description: First XCT measurements, 2024 metodology.


In [7]:
table_name = 'measurementtypes'

# Extract column names and values from the attributes dictionary
columns = ', '.join(main_parameters.keys())
values = ', '.join([f"'{v}'" for v in main_parameters.values()])

# Construct the SQL INSERT statement
sql = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"

print(sql)

# Create a cursor object using the cursor() method
cursor = conn.cursor()

# Execute the SQL statement
cursor.execute(sql)

cursor.execute('COMMIT')

cursor.close()

INSERT INTO measurementtypes (name, description) VALUES ('XCT_2024', 'First XCT measurements, 2024 metodology.')


## Make sure insert was correct

In [ ]:
data = qrs.get_data(table_name)

data

,id_measurementtype,name_measurementtype,description_measurementtype
0,1,UT_2025,"UT measurements, 2025 metodology. The ideal co..."
1,2,XCT_2024,"First XCT measurements, 2024 metodology."


Now we store the id of the inserted row, to insert its related metadata

In [9]:
row_id = data['id_' + table_name[:-1]].values[-1]

print(f"Row ID: {row_id}")

Row ID: 2


## Load into the metadata table

In [10]:
metadata_table_name =  table_name[:-1] + '_metadata'

for attributes in metadata_parameters:

    attributes[table_name[:-1] + '_id'] = row_id

    # Extract column names and values from the attributes dictionary
    columns = ', '.join(attributes.keys())
    values = ', '.join([f"'{v}'" for v in attributes.values()])

    # Construct the SQL INSERT statement
    sql = f"INSERT INTO {metadata_table_name} ({columns}) VALUES ({values})"

    print(sql)

    # Create a cursor object using the cursor() method
    cursor = conn.cursor()

    # Execute the SQL statement
    cursor.execute(sql)

    cursor.execute('COMMIT')

    cursor.close()

INSERT INTO measurementtype_metadata (key, value, type, measurementtype_id) VALUES ('technique', 'X Ray Computed Tomography', 'nominal', '2')
INSERT INTO measurementtype_metadata (key, value, type, measurementtype_id) VALUES ('voltage', '50', 'kV', '2')
INSERT INTO measurementtype_metadata (key, value, type, measurementtype_id) VALUES ('current', '90', 'uA', '2')
INSERT INTO measurementtype_metadata (key, value, type, measurementtype_id) VALUES ('resolution', '20', 'um', '2')
INSERT INTO measurementtype_metadata (key, value, type, measurementtype_id) VALUES ('frames', '5', 'cardinal', '2')
INSERT INTO measurementtype_metadata (key, value, type, measurementtype_id) VALUES ('exposure time', '0.2', 's', '2')
INSERT INTO measurementtype_metadata (key, value, type, measurementtype_id) VALUES ('projections', '3300', 'cardinal', '2')
INSERT INTO measurementtype_metadata (key, value, type, measurementtype_id) VALUES ('target', 'W', 'nominal', '2')
INSERT INTO measurementtype_metadata (key, val

## Make sure insert was correct

In [ ]:
data = qrs.get_data_metadata(table_name)

data

,id_measurementtype,name_measurementtype,description_measurementtype,technique_measurementtype,Nominal Frecuency_measurementtype,Z Resolution_measurementtype,X Resolution_measurementtype,Y Resolution_measurementtype,sample transducer distance_measurementtype,transducer z position_measurementtype,...,voltage_measurementtype,current_measurementtype,resolution_measurementtype,frames_measurementtype,exposure time_measurementtype,projections_measurementtype,target_measurementtype,filter_measurementtype,binning_measurementtype,equipment_measurementtype
0,1,UT_2025,"UT measurements, 2025 metodology. The ideal co...",Ultrasound Pulse Echo nominal,5 Mhz,0.02232141 mm,1 mm,1 mm,77 mm,120 mm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,XCT_2024,"First XCT measurements, 2024 metodology.",X Ray Computed Tomography nominal,NaN,NaN,NaN,NaN,NaN,NaN,...,50 kV,90 uA,20 um,5 cardinal,0.2 s,3300 cardinal,W nominal,LE1 nominal,1 cardinal,Versa nominal


In [12]:
conn.close()